## We Watch, We Listen, and We Control

House Micaeli understands the value of knowledge, and we use our magically-enhanced powers of perception to notice secrets that no normal man could. We have acted as detectives, inquisitors, and spy catchers, amassing power through observation.

In most cases, this has been observation of other people. A Micaeli's sharp senses can easily detect when a person is lying or pick up on a whispered conversation in a crowded marketplace. We can do this thanks to the eldritch tongue, which can be used to modify sound.

(cf: should we open with the spectrogram instead because it's cooler?)

In [ ]:
human_sample_one = get_first_example()
human_sample_two = get_second_example()

In [ ]:
listen(human_sample_one)

(You can change the above cell and re-run it to listen to the second sample.)

Micaelis can manipulate sound by representing it as a list of numbers -- indeed, sound itself is a series of pressure changes that propagates through the air. If we record a "pushing" pressure with a positive number and a "pulling" pressure with a negative number, then such a list can completely capture what your ear feels when someone speaks to you.

In [ ]:
print human_sample_one

But looking at such a set of numbers is of little use to us. We wish to use this data to learn something about the speaker, and use this for our own purposes. To start, a Micaeli can easily see the sound by plotting the sizes of the numbers in this list.

In [ ]:
plot(human_sample_one)

At least we can see which part of the sample is the loudest (has the highest pressure), but we can do better.

You will rightfully be impatient. Our House struggles for its very existence against the vicious aberrant invaders, while power over the City of Towers will soon change hands -- hopefully to ours.

The other barons of Micaeli have proposed a plan, and we seek your approval. Just as our House has come to power through investigating other people, learning their secrets, and using this as power, __what if we can prevail over the monstrous aberrants by observing them__? We must know our enemy, and if we can understand how these creatures communicate as well as we understand the speech of men, we will have the upper hand.

In [ ]:
aberrant_example = get_aberrant_example()

## Wrap a function here to plot all three vertically with shared x

__Challenge__. Use the listen function to observe the aberrant sound.

This will be a challenge even for us. These creatures sound so different from humans!

__Brainstorm__. With the other barons of your house, brainstorm as many characteristics of sound as you can think of. What properties does sound have which might differ between men and aberrants? Feel free to include physical properties, descriptions, or whatever else comes to mind.

Demo of how we can calculate features, including spectrogram if not done above.

Longer group challenge -- construct features from sound array. Give them examples of zero-crossing rate and energy. Ask them to think of other things and implement them. This will be scored.

First learning task -- let them listen to queen_acknowledgement1.wav and queen_annoyance3.wav. Explain the latter is annoyed. Then give them queen_annoyance2.wav. Ask them if it sounds annoyed or not? Hard to tell, but it is!

Reveal the secret -- we suspect that they're intelligent and want to try communicating with them. Want to create a translator using eldritch speech. First step is getting basic emotion, so need to use the eldritch tongue to understand whether they are angry or not. Challenge: write a program that takes in a sample of aberrant audio and outputs True if you guess it's angry, or False otherwise.

Score and show them the results. It'll probably be bad, like 0 or 1 correct. 

Probably around here the side quest will be introduced.